# [Module 3.2] Custom PCA Docker Image 생성 및 ECR Model 학습 

이 노트뷱은 Bring Your Own Container(BYOC)를 위해서 Custom Docker Image를 생성 합니다. 이 docker image는 학습 및 추론에 사용 됩니다.

구체적으로 이 노트북은 아래와 같은 작업을 합니다.

- Custom docker image name 정의
- PCA 학습 코드를 docker container 폴더로 복사
- Dockerfile 작성
- Docker Image 빌드 및 ECR에 등록
- Docker Image에 권한 부여
- Custom Docker Image 이름 저장
---
소요 시간은 약 5분 걸립니다.

In [1]:
import sagemaker
import pandas as pd
import numpy as np
import os
import time
import json
from time import strftime, gmtime

In [2]:
%store -r

## Custom docker image name 정의

Custom docker image 이름을 작성 하고, 필요한 account_id, region, bucket name 등을 가져 옵니다.

In [3]:
import boto3
import sagemaker
from sagemaker import get_execution_role

# Define custom docker image name
ecr_namespace = 'sagemaker-training-containers/'
prefix = 'pca'
ecr_repository_name = ecr_namespace + prefix
print("ecr_repository_name: ", ecr_repository_name)

role = get_execution_role()

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print("account_id: ", account_id)
print("region: ", region)
print("role: ", role)
print("bucket: ", bucket)

ecr_repository_name:  sagemaker-training-containers/pca
account_id:  057716757052
region:  ap-northeast-2
role:  arn:aws:iam::057716757052:role/service-role/AmazonSageMaker-ExecutionRole-20191128T110038
bucket:  sagemaker-ap-northeast-2-057716757052


## PCA 학습 코드를 docker container 폴더로 복사

이전에 작성한 PCA 학습 코드를 편의상 docker 폴더에 복사 함.

In [4]:
! cp pca_byoc_train.py docker/code/

## Dockerfile 작성

현재 Dockerfile은 SageMkaer built-in SKLearn image 를 Base로 하여 작성 하였습니다. <br>
```FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3```

위의 이미지는 <font color="red">만일 현재 Region이 ap-northwest-2 가 이니시면 반드시 해당 Region으로 변경 해주셔야 합니다.</font><br>
- 예: 현재 Ohio 인 경우 (us-east-2) 로 변경.
```
FROM 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
```

Region 마다 SKLearn Built-in docker image의 정보는 아래 URL을 참조하세요.

Prebuilt Amazon SageMaker Docker Images for Scikit-learn and Spark ML
- https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-docker-containers-frameworks.html

In [5]:
%%writefile docker/Dockerfile

FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
    
# install python package
RUN pip install joblib


ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

ENV PATH="/opt/ml/code:${PATH}"

# Copy training code
COPY code/* /opt/ml/code/
 
WORKDIR /opt/ml/code

# In order to use SageMaker Env varaibles, use the statement below
ENV SAGEMAKER_PROGRAM pca_byoc_train.py

Overwriting docker/Dockerfile


## Docker Image 빌드 및 ECR에 등록

Bash shell에서 사용할 변수를 아래와 같이 저장 합니다.

In [6]:
import os
os.environ['account_id'] = account_id
os.environ['region'] = region
os.environ['ecr_repository_name'] = ecr_repository_name

아래 셀은 다음과 같은 작업을 합니다.
- 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3 docker image를 가져오기 위해서 ECR의 366743142698 계정에 로긴 합니다.
- 위에서 정의한 Dockerfile로 docker image를 로컬에서 생성 합니다.
- docker image를 sagemaker-training-containers/pca 로 태깅 합니다.
- ECR의 나의 계정 (예: 057716757052)으로 로긴
- 현재 나의 계정에 해당 repository(예: sagemaker-training-containers/pca)가 존재하는지 확인 합니다.만일 존재 하지 않으면 새로이 생성 합니다
- ECR에 docker image를 등록 합니다.


**[중요]** 만일 Dockerfile에 명시된 
```FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3```
<font color="red">account_id (366743142698) 가 아래의 account_id(366743142698) 와 다르면 같게 만들어 주세요.</font>
```
$(aws ecr get-login --registry-ids 366743142698 --region ${region} --no-include-email)
```

In [7]:
%%sh

ACCOUNT_ID=${account_id}
REGION=${region}
REPO_NAME=${ecr_repository_name}

echo $REGION
echo $ACCOUNT_ID
echo $REPO_NAME


# Get the login command from ECR in order to pull down sagemaker-scikit-learn:0.20.0-cpu-py3 image
# sagemaker-scikit-learn:0.20.0-cpu-py3 docker image를 가져오기 위해서 ECR의 366743142698 계정에 로긴 합니다.
$(aws ecr get-login --registry-ids 366743142698 --region ${region} --no-include-email)


# Build a docker image in local
docker build -f docker/Dockerfile -t $REPO_NAME docker

# Tag an image referenced by Name ecr_repository_name(sagemaker-training-containers/pca)
docker tag $REPO_NAME $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest

# Get the login command for my account_id
# ECR의 나의 계정 (예: 057716757052)으로 로긴
$(aws ecr get-login --no-include-email --registry-ids $ACCOUNT_ID)

# 현재 나의 계정에 해당 repository(예: sagemaker-training-containers/pca) 존재하는지 확인 합니다.
# 만일 존재 하지 않으면 새로이 생성 합니다
aws ecr describe-repositories --repository-names $REPO_NAME || aws ecr create-repository --repository-name $REPO_NAME

# ECR에 docker image를 등록 합니다.
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest



ap-northeast-2
057716757052
sagemaker-training-containers/pca
Login Succeeded
Sending build context to Docker daemon  11.26kB
Step 1/8 : FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
 ---> 30adb1aa9af5
Step 2/8 : RUN pip install joblib
 ---> Using cache
 ---> 59663d1629c3
Step 3/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 605f60b5674c
Step 4/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> b70c6ea0613d
Step 5/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 29a1bfe2b5d6
Step 6/8 : COPY code/* /opt/ml/code/
 ---> Using cache
 ---> 9c639fea665d
Step 7/8 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 457c17c81eed
Step 8/8 : ENV SAGEMAKER_PROGRAM pca_byoc_train.py
 ---> Using cache
 ---> 46f5ed0dedd4
Successfully built 46f5ed0dedd4
Successfully tagged sagemaker-training-containers/pca:latest
Login Succeeded
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:ap-northeast-2:057716757052:r

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



ECR에 등록한 custome docker image의 이름 출력

In [8]:
custom_pca_docker_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(custom_pca_docker_image_uri)

057716757052.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-training-containers/pca:latest


## Docker Image에 권한 부여

추후에 Inference Pipleline을 통해 한개의 SageMaker Model을 만들시에 ```057716757052.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-training-containers/pca:latest``` 를 가져옵니다. 이때 Inference Pipeline을 실행 하는 주체는 sagemaker.amazonaws.com 입니다. 그래서 sagemaker.amazonaws.com에게 이 docker image를 가져올 수 있는 권한을 아래와 같이 할당 합니다.

아래와 같은 권한 할당이 안되어 있으면 아래와 같은 에러가 발생 합니다.
- HTTP 403: "User [arn] is not authorized to perform [operation]"
- 참조: https://docs.aws.amazon.com/AmazonECR/latest/userguide/common-errors.html

ECR 콘솔로 이동을 하여 위에서 생성한 Docker Image를 선택 합니다.

![Fig.3.2.ECR-Repository](img/Fig.3.2.ECR-Repository.png)

왼쪽의 permission 을 클릭하고, 오른쪽 상단에 "Edit policy JSON"을 클릭 합니다.

![Fig.3.2.ECR-permission](img/Fig.3.2.ECR-permission.png)

아래의 "Edit JSON"에 아래의 JSON 코드를 복사하여 붙입니다.
sagemaker.amazonaws.com 이 docker image의 특정 액션을 수행하게 허가 합니다.
```
{
  "Version": "2008-10-17",
  "Statement": [
    {
      "Sid": "allowSageMakerToPull",
      "Effect": "Allow",
      "Principal": {
        "Service": "sagemaker.amazonaws.com"
      },
      "Action": [
        "ecr:BatchCheckLayerAvailability",
        "ecr:BatchGetImage",
        "ecr:GetDownloadUrlForLayer"
      ]
    }
  ]
}
```

![Fig.3.2.ECR-edit-json](img/Fig.3.2.ECR-edit-json.png)

아래와 같은 화면이 나오면 완료 입니다.
![Fig.3.2.ECR-permission-finish](img/Fig.3.2.ECR-permission-finish.png)

## Custom Docker Image 이름 저장

In [9]:
%store custom_pca_docker_image_uri

Stored 'custom_pca_docker_image_uri' (str)
